In [1]:
import pandas as pd

In [2]:
import pandas as pd

data_df = pd.read_csv(r'/content/dataset - cleaned_hm.csv')
data_df.head()
data = data_df

In [3]:
rows =  data.loc[data['num_sentence'] > 10]
mod_data = rows["cleaned_hm"].to_list()
mod_data[1]

'Sunset is the time when the sun goes down and night begins. During the sunset, nature assumes fresh, bright, and charming. The sun seems to be a disc of gold. At the time of sunset, one can enjoy the serenity of a sunset. Then nature seems in a hurry. People finish their work and return home. Birds return to their nests. The cowboys become busy with returning home with cattle. When the sun sets, nature looks very beautiful. Then the nature starts getting dark. The slight darkness of the sunset creates a feeling of joy and tranquility in us. The beauty of the sunset helps a poet to generate any ideas and compose poems.'

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

punctuation_set = set(string.punctuation)

def preprocess_sentence(sentence):

    words = word_tokenize(sentence)

    words = [word for word in words if word not in punctuation_set]

    processed_sentence = ' '.join(words)
    return processed_sentence

def preprocess_data(data):
    processed_data = []
    for paragraph in data:

        sentences = sent_tokenize(paragraph)

        processed_sentences = [preprocess_sentence(sentence) for sentence in sentences]

        processed_data.extend(processed_sentences)
    return processed_data

processed_data = preprocess_data(mod_data)
print(processed_data[0])

This is the second essay in a two-part series about my journey to visit the Taj Mahal


PART A

In [6]:
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from collections import defaultdict

tokenized_dataset = [sentence.split() for sentence in processed_data]

model = Word2Vec(sentences=tokenized_dataset, min_count=1, vector_size=100, window=5)

def predict_context(model, word):
    return model.wv.most_similar(word)

word_to_predict = "someone"
similar_words = predict_context(model, word_to_predict)
similar_words

[('or', 0.9989322423934937),
 ('its', 0.998910129070282),
 ('all', 0.9989023208618164),
 ('when', 0.9989014863967896),
 ('us', 0.998879611492157),
 ('from', 0.9988564848899841),
 ('He', 0.9988465905189514),
 ('were', 0.9988460540771484),
 ('who', 0.9988430738449097),
 ('after', 0.9988406896591187)]

In [7]:
len(processed_data)

4475

PART B

In [8]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Tokenize the sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(processed_data)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
X, y = input_sequences[:,:-1],input_sequences[:,-1]
y = to_categorical(y, num_classes=total_words)

# Define the model
embedding_dim = 50
model = Sequential()
model.add(Embedding(total_words, embedding_dim, input_length=max_sequence_len-1))
model.add(SimpleRNN(100))
model.add(Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='loss', patience=3)
model.fit(X, y, epochs=50, verbose=1, callbacks=[early_stopping])

Epoch 1/50
1/1 [==============================] - 1s 1s/step - loss: 8.6080 - accuracy: 0.0000e+00
Epoch 2/50
1/1 [==============================] - 0s 28ms/step - loss: 8.6000 - accuracy: 0.0000e+00
Epoch 3/50
1/1 [==============================] - 0s 21ms/step - loss: 8.5917 - accuracy: 0.0000e+00
Epoch 4/50
1/1 [==============================] - 0s 20ms/step - loss: 8.5833 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 22ms/step - loss: 8.5746 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 20ms/step - loss: 8.5656 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 24ms/step - loss: 8.5561 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 23ms/step - loss: 8.5461 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 24ms/step - loss: 8.5354 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 28ms/step - loss: 8.5240 - accuracy: 1.0000
Epoch 11/50
1/1 [==

In [9]:
# Function to generate next and previous words
def generate_next_and_prev_word(seed_text):
    for _ in range(2): # Predict both next and previous word
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        print(token_list)
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Example usage
seed_text = "Taj"
predicted_text = generate_next_and_prev_word(seed_text)
print(predicted_text)

[271]
1/1 [==============================] - 0s 255ms/step
[271, 1716]
1/1 [==============================] - 0s 20ms/step
Taj period truth
